In [27]:
#--Librerias--#
import pandas as pd 
import numpy as np
from sklearn.model_selection import RandomizedSearchCV # búsqueda aleatoria de hiperparámetros 
import joblib  ### para guardar y cargar modelos
from sklearn.preprocessing import StandardScaler ## escalar variables 
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Regresión logística
from sklearn.ensemble import RandomForestClassifier  # Clasificador bosques aleatoriost 
from sklearn.tree import DecisionTreeClassifier # Arboles de decision 
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler ## escalar variable
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier #Modelo de clasificacion
from sklearn.metrics import confusion_matrix #### Matriz de confusion 
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns #####Graficos
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import funciones


In [15]:
###----Lectura de datos----###
df = pd.read_csv("https://raw.githubusercontent.com/andressj1/Predicci-n-precio-de-m-viles-/refs/heads/main/Datos/df_fin", sep=",")

# Dummies y Escalar variables

In [16]:
# Conversion; categoricas a dummies
df = pd.get_dummies(df, dtype = int)

In [17]:
# Separamos variables explicativas - variable objetivo
X1 = df.drop('price_range',axis=1)
y = df['price_range']

print(X1.shape)
print(y.shape)

(1958, 20)
(1958,)


In [18]:
# Escalar variables 
scaler = StandardScaler()
scaler.fit(X1)
X2 = scaler.transform(X1)
X = pd.DataFrame(X2,columns=X1.columns)

# Selección de variables

*1. Selec From Model*


*1.1. Ridge Classifier*

In [19]:
# Selector de variables con Ridge
sel_ = SelectFromModel(RidgeClassifier(alpha = 0.1, random_state = 24), max_features = 20)
sel_.fit(X, y)
print(sel_.estimator_.coef_)
#Obtener variables seleccionadas
X_new = sel_.get_support()

df_ridge = X.iloc[:,X_new]
df_ridge.info()

[[-0.14807826 -0.00397091 -0.0037951  -0.00743636  0.0082613  -0.00317897
  -0.02776882  0.02098687 -0.1180061  -0.02613334  0.00751121 -0.0039677
   0.00401115  0.01599757  0.00955429  0.33717986 -0.19710618 -0.18095961
   0.0042275  -0.00213541]
 [-0.01447262  0.00444907  0.01678706  0.00346025  0.01086592  0.00909252
   0.03465646 -0.04806438 -0.00368244  0.02971434  0.00959173  0.00253166
  -0.01931998 -0.0359268  -0.02646579  0.15950893  0.02111461 -0.20201974
   0.1078598   0.10489199]
 [ 0.01400606 -0.00395575 -0.00684944  0.01600108 -0.03175667 -0.02774011
  -0.0021118   0.02327536 -0.0064283  -0.01048074 -0.03263826 -0.00814626
  -0.00549342  0.00164768  0.02715273 -0.20932219  0.31927867 -0.08811182
  -0.1174903  -0.0663456 ]
 [ 0.14854482  0.0034776  -0.00614252 -0.01202496  0.01262945  0.02182655
  -0.00477584  0.00380215  0.12811685  0.00689973  0.01553533  0.0095823
   0.02080226  0.01828154 -0.01024123 -0.2873666  -0.1432871   0.47109116
   0.00540301 -0.03641098]]
<clas

*1.2. Gradient Boosting*

In [20]:
# Selector de variables con Gradient Boosting
sel_ = SelectFromModel(GradientBoostingClassifier(n_estimators = 300, random_state=24))
sel_.fit(X, y)
print(sel_.estimator_.feature_importances_)
#Obtener variables seleccionadas
X_new = sel_.get_support()

df_gb = X.iloc[:,X_new]
df_gb.head()

[0.17474066 0.00111653 0.0014376  0.01537186 0.00322616 0.03021425
 0.01002424 0.00940855 0.13469667 0.02246023 0.00261395 0.00101846
 0.00086092 0.00335158 0.00246889 0.18001534 0.12125409 0.27971146
 0.00225737 0.00375119]


,battery_power,px_width,ram_cat_2GB,ram_cat_3GB,ram_cat_4GB
0,-0.900357,-1.146484,-0.920418,1.610174,-0.597407
1,-0.492840,1.707028,-0.920418,1.610174,-0.597407
2,-1.535536,1.077032,-0.920418,1.610174,-0.597407
3,-1.417151,1.239164,-0.920418,1.610174,-0.597407
4,1.328463,-0.090314,1.086463,-0.621051,-0.597407


# Prueba de selecciones

Ridge vs RL

In [28]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test,y_train,y_test = train_test_split(df_ridge,y,test_size=0.2, random_state=42)

# Entrenar el modelo
lr = LogisticRegression(max_iter=1000,class_weight="balanced", random_state=42).fit(X_train,y_train)

# Predicciones
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
y_pred_prob_train = lr.predict_proba(X_train)
y_pred_prob_test = lr.predict_proba(X_test)

# Matriz de confusión
mc_train = confusion_matrix(y_train, y_pred_train)
mc_test = confusion_matrix(y_test, y_pred_test)

# Métricas para los datos de entrenamiento
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')
accuracy_train = accuracy_score(y_train, y_pred_train)

# Métricas para los datos de prueba
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
accuracy_test = accuracy_score(y_test, y_pred_test)

# Imprimir los resultados de entrenamiento
print('-'*30, 'TRAIN', '-'*30)
print(f'Precision (train): {precision_train}')
print(f'Recall (train): {recall_train}')
print(f'F1 score (train): {f1_train}')
print('Train score (accuracy): ', accuracy_train)

# Imprimir los resultados de prueba
print('-'*30, 'TEST', '-'*30)
print(f'Precision (test): {precision_test}')
print(f'Recall (test): {recall_test}')
print(f'F1 score (test): {f1_test}')
print('Test score (accuracy): ', accuracy_test)


------------------------------ TRAIN ------------------------------
Precision (train): 0.6892352506136701
Recall (train): 0.6954022988505747
F1 score (train): 0.6901928737717559
Train score (accuracy):  0.6954022988505747
------------------------------ TEST ------------------------------
Precision (test): 0.7059224885499565
Recall (test): 0.7091836734693877
F1 score (test): 0.7023819871308545
Test score (accuracy):  0.7091836734693877


Ridge vs Random Forest

In [31]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test,y_train,y_test = train_test_split(df_ridge,y,test_size=0.2, random_state=42)

# Entrenar el modelo
ranfor = RandomForestClassifier(n_estimators=300,
                                  max_depth=30,
                                  n_jobs=-1,
                                  max_leaf_nodes=20,
                                  min_samples_leaf=10,
                                  class_weight="balanced", random_state=42).fit(X_train,y_train)


# Predicciones
y_pred_train = ranfor.predict(X_train)
y_pred_test = ranfor.predict(X_test)
y_pred_prob_train = ranfor.predict_proba(X_train)
y_pred_prob_test = ranfor.predict_proba(X_test)

# Matriz de confusión
mc_train = confusion_matrix(y_train, y_pred_train)
mc_test = confusion_matrix(y_test, y_pred_test)

# Métricas para los datos de entrenamiento
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')
accuracy_train = accuracy_score(y_train, y_pred_train)

# Métricas para los datos de prueba
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
accuracy_test = accuracy_score(y_test, y_pred_test)

# Imprimir los resultados de entrenamiento
print('-'*30, 'TRAIN', '-'*30)
print(f'Precision (train): {precision_train}')
print(f'Recall (train): {recall_train}')
print(f'F1 score (train): {f1_train}')
print('Train score (accuracy): ', accuracy_train)

# Imprimir los resultados de prueba
print('-'*30, 'TEST', '-'*30)
print(f'Precision (test): {precision_test}')
print(f'Recall (test): {recall_test}')
print(f'F1 score (test): {f1_test}')
print('Test score (accuracy): ', accuracy_test)


------------------------------ TRAIN ------------------------------
Precision (train): 0.7249309317194478
Recall (train): 0.7266922094508301
F1 score (train): 0.7139822507691984
Train score (accuracy):  0.7266922094508301
------------------------------ TEST ------------------------------
Precision (test): 0.7006159569365886
Recall (test): 0.701530612244898
F1 score (test): 0.6871501889531658
Test score (accuracy):  0.701530612244898


Gradient Boosting vs RL

In [29]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test,y_train,y_test = train_test_split(df_gb,y,test_size=0.2, random_state=42)

# Entrenar el modelo
lr = LogisticRegression(max_iter=1000,class_weight="balanced", random_state=42).fit(X_train,y_train)

# Predicciones
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
y_pred_prob_train = lr.predict_proba(X_train)
y_pred_prob_test = lr.predict_proba(X_test)

# Matriz de confusión
mc_train = confusion_matrix(y_train, y_pred_train)
mc_test = confusion_matrix(y_test, y_pred_test)

# Métricas para los datos de entrenamiento
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')
accuracy_train = accuracy_score(y_train, y_pred_train)

# Métricas para los datos de prueba
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
accuracy_test = accuracy_score(y_test, y_pred_test)

# Imprimir los resultados de entrenamiento
print('-'*30, 'TRAIN', '-'*30)
print(f'Precision (train): {precision_train}')
print(f'Recall (train): {recall_train}')
print(f'F1 score (train): {f1_train}')
print('Train score (accuracy): ', accuracy_train)

# Imprimir los resultados de prueba
print('-'*30, 'TEST', '-'*30)
print(f'Precision (test): {precision_test}')
print(f'Recall (test): {recall_test}')
print(f'F1 score (test): {f1_test}')
print('Test score (accuracy): ', accuracy_test)

------------------------------ TRAIN ------------------------------
Precision (train): 0.6923671737812298
Recall (train): 0.6979565772669221
F1 score (train): 0.6929323152097137
Train score (accuracy):  0.6979565772669221
------------------------------ TEST ------------------------------
Precision (test): 0.7105004269010194
Recall (test): 0.7142857142857143
F1 score (test): 0.7066961613938635
Test score (accuracy):  0.7142857142857143


Gradient Boosting vs Random Forest

Todas vs Random Forest

Todas vs RL

In [30]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

# Entrenar el modelo
lr = LogisticRegression(max_iter=1000,class_weight="balanced", random_state=42).fit(X_train,y_train)

# Predicciones
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
y_pred_prob_train = lr.predict_proba(X_train)
y_pred_prob_test = lr.predict_proba(X_test)

# Matriz de confusión
mc_train = confusion_matrix(y_train, y_pred_train)
mc_test = confusion_matrix(y_test, y_pred_test)

# Métricas para los datos de entrenamiento
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')
accuracy_train = accuracy_score(y_train, y_pred_train)

# Métricas para los datos de prueba
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
accuracy_test = accuracy_score(y_test, y_pred_test)

# Imprimir los resultados de entrenamiento
print('-'*30, 'TRAIN', '-'*30)
print(f'Precision (train): {precision_train}')
print(f'Recall (train): {recall_train}')
print(f'F1 score (train): {f1_train}')
print('Train score (accuracy): ', accuracy_train)

# Imprimir los resultados de prueba
print('-'*30, 'TEST', '-'*30)
print(f'Precision (test): {precision_test}')
print(f'Recall (test): {recall_test}')
print(f'F1 score (test): {f1_test}')
print('Test score (accuracy): ', accuracy_test)

------------------------------ TRAIN ------------------------------
Precision (train): 0.713364183321103
Recall (train): 0.7158365261813537
F1 score (train): 0.7116404374077976
Train score (accuracy):  0.7158365261813537
------------------------------ TEST ------------------------------
Precision (test): 0.7025776055631978
Recall (test): 0.7066326530612245
F1 score (test): 0.6969587496518522
Test score (accuracy):  0.7066326530612245
